In [17]:
__author__ = 'Daisuke Yoda'
__Date__ = 'December 2018'

## Loading Modules

In [18]:
import warnings
warnings.filterwarnings('ignore')

from collections import defaultdict
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np
from copy import deepcopy
from collections import Counter
from itertools import combinations
import pandas as pd
import os
__dir__ = os.getcwd()[:-11]

from gensim.models.keyedvectors import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format(__dir__ + 'data/glove.6B.100d.bin')

## Functions 

In [4]:
def make_pairs(words, max_len =6):
    patterns = defaultdict(list)

    for word in words:
        for second_word in words:
            if word != second_word:
                i = 1
                while(word[:i]==second_word[:i]):
                    i += 1
                if i != 1 and i > max(len(word[i-1:]), len(second_word[i-1:])) < max_len:
                    if ("suffix", word[i-1:], second_word[i-1:]) in patterns:
                        patterns[("suffix", word[i-1:], second_word[i-1:])].append((word, second_word))
                    else:
                        patterns[("suffix", word[i-1:], second_word[i-1:])] = [(word, second_word)]
                        
                i = 1
                while(word[-i:]==second_word[-i:]):
                    i += 1
                if i != 1 and max(len(word[:-i+1]), len(second_word[:-i+1])) < max_len:
                    if ("prefix", word[:-i+1], second_word[:-i+1]) in patterns:
                        patterns[("prefix", word[:-i+1], second_word[:-i+1])].append((word, second_word))
                    else:
                        patterns[("prefix", word[:-i+1], second_word[:-i+1])] = [(word, second_word)]
                        
    return patterns


def molph_classify(thepairs,model,threshold=0.5,min_category=5):
    new_pairs = defaultdict(list)

    for key in thepairs:
        cadidates = thepairs[key]
        
        similality = []
        for pair in cadidates:
            try:
                cos_sim = model(pair[0],pair[1])
            except:
                pass
            else: 
                if cos_sim > threshold:
                    similality.append(pair + (cos_sim,))
        
        if len(similality) > min_category :
             new_pairs[key] = similality
             
    return new_pairs


def make_same_group(pairs,word):
    pair_list = sum(list(pairs.values()),[])
    group = [pair for pair in pair_list if word==(pair[0] or pair[1])]
            
    return group


def plot_graph(pair_group):
    G = nx.Graph()
    for pair in pair_group:
        G.add_nodes_from([pair[0],pair[1]])
        G.add_edge(pair[0],pair[1])
    
    plt.figure(figsize=(7,7))
    pos = nx.spring_layout(G,k=0.7)
    nx.draw_networkx_nodes(G, pos, node_color='lightgray', node_size=5000.0)
    nx.draw_networkx_edges(G, pos, width=2.5)
    nx.draw_networkx_labels(G, pos, fontsize=25, font_weight="bold")
    plt.axis('on')
    plt.show()
    

def get_hit_rate(pairs, words):
    hit_word = []
    pair_list = sum(list(pairs.values ()), [])
    for word in words:
        hit_word.extend([pair[1] for pair in pair_list if word == pair[0]])
        hit_word.extend([pair[0] for pair in pair_list if word == pair[1]])

    c = Counter(hit_word)
    
    return [(pair[0],int(pair[1]/2)) for pair in c.most_common(100)]
    

def clustering_group(pairs):
    pair_list1 = [set(pair[0:2]) for pair in sum(list(pairs.values()), [])]
    pair_list2 = deepcopy(pair_list1)
    cluster_list = []
    for pair1 in pair_list1[:]:
        ald = False
        for cluster in cluster_list:
            if pair1 <= cluster:
                ald = True
        if ald:
            continue

        while True:
            calc = False
            for pair2 in pair_list2[:]:
                if pair1 >= pair2:
                    pass
                elif not pair1.isdisjoint (pair2):
                    pair1 = pair1 | pair2
                    calc = True
            if not calc:
                cluster_list.append (pair1)
                break

    return cluster_list

def get_stem(cluster):
    stem_candidate = []
    for pair in combinations(cluster,2):
        stem_candidate.append([char1 for char1,char2 in zip(pair[0],pair[1]) if char1 == char2])

    stem = sorted(stem_candidate,key=lambda x: len(x))

    return ''.join(stem[0])


def vector_by_stem(stem_cluster,pd_frame=True):
    split_vec = {}
    for stem in stem_cluster.keys():
        for word in stem_cluster[stem]:
            if word[:len(stem)] == stem:
                vec = np.eye(len(word))[len(stem) - 1]


            elif word[::-1][:len(stem[::-1])] == stem[::-1]:
                vec = np.eye(len(word))[len(stem)][::-1]


            if pd_frame:
                split_vec[word] = pd.Series(vec)
            else:
                split_vec[word] = vec

    return pd.DataFrame(split_vec)

## The Dataset

In [5]:
with open(__dir__ + 'data/english_brown.txt') as f:
    data = f.read()
    data = data.replace('.','')
    data = data.replace(',','')
    data = data.replace('""','')
    data = data.lower()

all_words = data.split()
words_set = np.unique(all_words)
words_set = [word for word in words_set if word.isalpha()]
words_set[:5]

['a', 'aaron', 'aau', 'abandoned', 'abandonment']

## Model for comparing consine similalities

In [6]:
model = word_vectors.similarity

## Making cadidates with same suffix and prefix

In [24]:
original_pair = make_pairs(words_set, max_len =6)

## Caluculating the cosine similalities to eliminate the unrelated words

In [25]:
pairs = molph_classify(original_pair,model,threshold=0.7,min_category=5)
pairs[('suffix', 'y', 'ied')]

[('apply', 'applied', 0.7922291),
 ('carry', 'carried', 0.7707906),
 ('deny', 'denied', 0.7732344),
 ('justify', 'justified', 0.78390497),
 ('qualify', 'qualified', 0.7768942),
 ('study', 'studied', 0.70555717),
 ('testify', 'testified', 0.74732226),
 ('vary', 'varied', 0.7457148)]

## Clustering

In [22]:
morph_cluster = clustering_group(pairs)
morph_cluster = [pair for pair in morph_cluster if len(pair) > 2]
morph_cluster = [sorted(pair,key=lambda x: len(x)) for pair in morph_cluster]
morph_cluster[:10]

[['age', 'aged', 'ages'],
 ['agree', 'agreed', 'agreeing', 'agreement', 'agreements'],
 ['announce', 'announced', 'announcing', 'announcement'],
 ['appreciate', 'appreciated', 'appreciation'],
 ['assure', 'assured', 'assuring'],
 ['believe', 'believed', 'believes', 'believing'],
 ['birdie', 'birdied', 'birdies'],
 ['cause', 'causes', 'caused'],
 ['celebrate', 'celebrates', 'celebrated', 'celebrating', 'celebration'],
 ['change', 'changed', 'changes', 'changing']]

## Get stem word to generate split point data

In [20]:
stem_cluster = {get_stem(cluster):cluster for cluster in morph_cluster}
split_list = vector_by_stem(stem_cluster)
split_list.T.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
age,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aged,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ages,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
agree,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
agreed,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
